In [2]:
from mock_generator import run_mock_simulation
import matplotlib.pyplot as plt
import multiprocessing as mp

In [3]:
mp.cpu_count()  # 检查 CPU 核心数

12

In [4]:
n_samples = 100000
mock_lens_data, mock_observed_data = run_mock_simulation(n_samples, process=8)
print(mock_lens_data.head())
print(mock_observed_data.head())

Processing lenses (process=8): 100%|██████████| 100000/100000 [00:00<00:00, 10659780.92it/s]


          xA         xB      beta    kappaA    kappaB    gammaA    gammaB  \
0  19.660818  -7.290712  0.739426  0.276033  1.022827  0.394218  0.866442   
1  29.581466 -15.741930  0.469573  0.261209  0.683599  0.458853  0.842462   
2  37.091554  -6.090340  0.964317  0.174823  2.158850  0.350063  1.734782   
4  33.143867  -9.686092  0.791406  0.202063  1.215155  0.383121  1.204295   
5   8.487310  -6.336645  0.222081  0.397016  0.574002  0.469345  0.605018   

   magnificationA  magnificationB  kappa_starA  ...  maximum_magnitude  \
0        2.712060        1.333058     0.009063  ...               26.5   
1        2.982685        1.640380     0.006758  ...               26.5   
2        1.790912        0.600131     0.001574  ...               26.5   
4        2.041136        0.712264     0.002797  ...               26.5   
5        6.977766        5.418402     0.082885  ...               26.5   

   beta_unit  logalpha_sps  logM_star  logM_star_sps  logM_star_sps_observed  \
0   0.739426

In [11]:
import numpy as np

# === 拼接所有数据 ===
kappa_all = np.concatenate([mock_lens_data['kappaA'], mock_lens_data['kappaB']])
gamma_all = np.concatenate([mock_lens_data['gammaA'], mock_lens_data['gammaB']])
s_all     = np.concatenate([mock_lens_data['sA'],     mock_lens_data['sB']])

# === 样本数组 ===
samples = np.vstack([kappa_all, gamma_all, s_all]).T

# === 立方体边界 ===
mins = samples.min(axis=0)
maxs = samples.max(axis=0)
print("[INFO] 立方体边界: ")
print(f"kappa: {mins[0]:.2f} ~ {maxs[0]:.2f}")
print(f"gamma: {mins[1]:.2f} ~ {maxs[1]:.2f}")
print(f"s:     {mins[2]:.2f} ~ {maxs[2]:.2f}")

# === 网格分箱数 ===
n_bins = 50  # 或更大，平衡精度与内存

# === 归一化到 [0, n_bins) ===
normed = (samples - mins) / (maxs - mins + 1e-10)
indices = np.floor(normed * n_bins).astype(int)
indices = np.clip(indices, 0, n_bins - 1)

# === 统计体素占用 ===
voxels = np.zeros((n_bins, n_bins, n_bins), dtype=bool)
for i in indices:
    voxels[i[0], i[1], i[2]] = True

# === 有效体素比例 ===
occupied = np.sum(voxels)
total = n_bins ** 3
fraction = occupied / total

print(f"[RESULT] 有效体素占比: {occupied}/{total} = {fraction:.6f}")


[INFO] 立方体边界: 
kappa: 0.08 ~ 7.79
gamma: 0.27 ~ 6.88
s:     0.09 ~ 1.00
[RESULT] 有效体素占比: 3125/125000 = 0.025000


In [5]:
mock_lens_data.columns

Index(['xA', 'xB', 'beta', 'kappaA', 'kappaB', 'gammaA', 'gammaB',
       'magnificationA', 'magnificationB', 'kappa_starA', 'kappa_starB',
       'alphaA', 'alphaB', 'sA', 'sB', 'einstein_radius_kpc',
       'einstein_radius_arcsec', 'scatter_mag', 'is_lensed',
       'magnitude_observedA', 'magnitude_observedB', 'mag_source',
       'maximum_magnitude', 'beta_unit', 'logalpha_sps', 'logM_star',
       'logM_star_sps', 'logM_star_sps_observed', 'logM_halo', 'logRe', 'zl',
       'zs'],
      dtype='object')

In [6]:
kappaA  = mock_lens_data['kappaA']
kappaB  = mock_lens_data['kappaB']
gammaA  = mock_lens_data['gammaA']
gammaB  = mock_lens_data['gammaB']
sA      = mock_lens_data['sA']
sB      = mock_lens_data['sB']

In [ ]:
def plot_histograms(data, ax, title, bins):
    ax.hist(data, bins=bins, alpha=0.7)
    ax.set_title(title)
    ax.set_xlabel('Value')
    ax.set_ylabel('Frequency')
    # ax.grid(True)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()
bins = 60
plot_histograms(kappaA, axes[0], 'Kappa A Distribution', bins=bins)
plot_histograms(gammaA, axes[1], 'Gamma A Distribution', bins=bins)
plot_histograms(sA, axes[2], 'S A Distribution', bins=bins)
plot_histograms(kappaB, axes[3], 'Kappa B Distribution', bins=bins)
plot_histograms(gammaB, axes[4], 'Gamma B Distribution', bins=bins)
plot_histograms(sB, axes[5], 'S B Distribution', bins=bins)


In [ ]:
plt.figure(figsize=(15, 10), dpi=100)
plt.scatter(kappaA, gammaA, alpha=0.5, label='A', s=10, marker='o')
plt.scatter(kappaB, gammaB, alpha=0.5, label='B', s=10, marker='x')
plt.title('Kappa vs Gamma')
plt.xlabel('Kappa')
plt.ylabel('Gamma')
plt.legend()
plt.grid(True)
plt.show()
# plt.savefig('kappa_vs_gamma.png', dpi=300, bbox_inches='tight')

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# === 样式和字体设置 ===
# plt.figure(figsize=(12, 8), dpi=300)
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300  # 或更高，如 300


sns.set_theme(style="white", font_scale=1.3)
sns.set_context("talk")  # 可选: "notebook", "talk", "paper", "poster"
palette = sns.color_palette("colorblind")[:2]  # 只取前两个颜色

# === 构造 DataFrame ===
df_A = pd.DataFrame({
    '$\kappa$': mock_lens_data['kappaA'],
    '$\gamma$': mock_lens_data['gammaA'],
    '$s$': mock_lens_data['sA'],
    'image': 'A'
})

df_B = pd.DataFrame({
    '$\kappa$': mock_lens_data['kappaB'],
    '$\gamma$': mock_lens_data['gammaB'],
    '$s$': mock_lens_data['sB'],
    'image': 'B'
})

df = pd.concat([df_A, df_B], ignore_index=True)

# === Pairplot 美化 ===
g = sns.pairplot(
    df,
    hue="image",
    corner=True,
    diag_kind="kde",  # 更光滑
    plot_kws=dict(alpha=0.1, s=15, edgecolor='none'),
    diag_kws=dict(fill=True),
    palette=palette,
    height=2.8,       # 每个子图大小
    aspect=1.0        # 宽高比，1 更对称
)

# === 获取 axes 对象 ===
axes = g.axes

# === 添加 κ=1.5 的竖线和横线 ===
# κ: index = 0
axes[1][0].axvline(1.5, color='black', linestyle='--', lw=1)  # γ vs κ
axes[2][0].axvline(1.5, color='black', linestyle='--', lw=1)  # s vs κ
axes[0][0].axvline(1.5, color='black', linestyle='--', lw=1)  # diag κ

# === 添加 γ=1.5 的横线和竖线 ===
# γ: index = 1
axes[1][0].axhline(1.5, color='black', linestyle='--', lw=1)  # γ vs κ
axes[2][1].axvline(1.5, color='black', linestyle='--', lw=1)  # s vs γ
axes[1][1].axvline(1.5, color='black', linestyle='--', lw=1)  # diag γ



# === 图例处理 ===
legend = g._legend
legend.set_title("Image")
legend.set_bbox_to_anchor((1, 1))  # 移到右上角之外
legend.set_frame_on(False)
for text in legend.texts:
    text.set_fontsize(12)

# === 调整布局 ===
plt.tight_layout(pad=1.5)  # pad 控制边距
plt.subplots_adjust(right=0.85)  # 给图例留出空间

plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# === 样式和字体设置 ===
# plt.figure(figsize=(12, 8), dpi=300)
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300  # 或更高，如 300


sns.set_theme(style="white", font_scale=1.3)
sns.set_context("talk")  # 可选: "notebook", "talk", "paper", "poster"
palette = sns.color_palette("colorblind")[:2]  # 只取前两个颜色

# === 构造 DataFrame ===
df_A = pd.DataFrame({
    '$\kappa$': mock_lens_data['kappaA'],
    '$\gamma$': mock_lens_data['gammaA'],
    '$s$': mock_lens_data['sA'],
    'image': 'A'
})

df_B = pd.DataFrame({
    '$\kappa$': mock_lens_data['kappaB'],
    '$\gamma$': mock_lens_data['gammaB'],
    '$s$': mock_lens_data['sB'],
    'image': 'B'
})

df = pd.concat([df_A, df_B], ignore_index=True)

# === Pairplot 美化 ===
g = sns.pairplot(
    df,
    hue="image",
    corner=True,
    diag_kind="kde",  # 更光滑
    plot_kws=dict(alpha=0.1, s=15, edgecolor='none'),
    diag_kws=dict(fill=True),
    palette=palette,
    height=2.8,       # 每个子图大小
    aspect=1.0        # 宽高比，1 更对称
)

# === 获取 axes 对象 ===
axes = g.axes

# === 添加 κ=1.5 的竖线和横线 ===
# κ: index = 0
axes[1][0].axvline(1.5, color='black', linestyle='--', lw=1)  # γ vs κ
axes[2][0].axvline(1.5, color='black', linestyle='--', lw=1)  # s vs κ
axes[0][0].axvline(1.5, color='black', linestyle='--', lw=1)  # diag κ

# === 添加 γ=1.5 的横线和竖线 ===
# γ: index = 1
axes[1][0].axhline(1.5, color='black', linestyle='--', lw=1)  # γ vs κ
axes[2][1].axvline(1.5, color='black', linestyle='--', lw=1)  # s vs γ
axes[1][1].axvline(1.5, color='black', linestyle='--', lw=1)  # diag γ



# === 图例处理 ===
legend = g._legend
legend.set_title("Image")
legend.set_bbox_to_anchor((1, 1))  # 移到右上角之外
legend.set_frame_on(False)
for text in legend.texts:
    text.set_fontsize(12)

# === 调整布局 ===
plt.tight_layout(pad=1.5)  # pad 控制边距
plt.subplots_adjust(right=0.85)  # 给图例留出空间

plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# === 提取变量 ===
kappaA = mock_lens_data['kappaA'].values
kappaB = mock_lens_data['kappaB'].values
gammaA = mock_lens_data['gammaA'].values
gammaB = mock_lens_data['gammaB'].values

# === 阈值范围 ===
thresholds = np.linspace(1.5, 2.0, 100)

# === 计算每个阈值下的比例 ===
def compute_ratio(arr, thresholds):
    return np.array([(arr > t).sum() / len(arr) for t in thresholds])

ratio_kappaA = compute_ratio(kappaA, thresholds)
ratio_kappaB = compute_ratio(kappaB, thresholds)
ratio_gammaA = compute_ratio(gammaA, thresholds)
ratio_gammaB = compute_ratio(gammaB, thresholds)

# === 可视化 ===
plt.figure(dpi=120)
plt.plot(thresholds, ratio_kappaA, label=r"$\kappa_A$")
plt.plot(thresholds, ratio_kappaB, label=r"$\kappa_B$")
plt.plot(thresholds, ratio_gammaA, label=r"$\gamma_A$")
plt.plot(thresholds, ratio_gammaB, label=r"$\gamma_B$")
plt.xlabel("Threshold")
plt.ylabel("Fraction > Threshold")
plt.title("Fraction of Values Exceeding Threshold")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np

# === 拼接所有数据 ===
kappa_all = np.concatenate([mock_lens_data['kappaA'], mock_lens_data['kappaB']])
gamma_all = np.concatenate([mock_lens_data['gammaA'], mock_lens_data['gammaB']])
s_all     = np.concatenate([mock_lens_data['sA'],     mock_lens_data['sB']])

# === 样本数组 ===
samples = np.vstack([kappa_all, gamma_all, s_all]).T

# === 立方体边界 ===
mins = samples.min(axis=0)
maxs = samples.max(axis=0)
print("[INFO] 立方体边界: ")
print(f"kappa: {mins[0]:.2f} ~ {maxs[0]:.2f}")
print(f"gamma: {mins[1]:.2f} ~ {maxs[1]:.2f}")
print(f"s:     {mins[2]:.2f} ~ {maxs[2]:.2f}")

# === 网格分箱数 ===
n_bins = 50  # 或更大，平衡精度与内存

# === 归一化到 [0, n_bins) ===
normed = (samples - mins) / (maxs - mins + 1e-10)
indices = np.floor(normed * n_bins).astype(int)
indices = np.clip(indices, 0, n_bins - 1)

# === 统计体素占用 ===
voxels = np.zeros((n_bins, n_bins, n_bins), dtype=bool)
for i in indices:
    voxels[i[0], i[1], i[2]] = True

# === 有效体素比例 ===
occupied = np.sum(voxels)
total = n_bins ** 3
fraction = occupied / total

print(f"[RESULT] 有效体素占比: {occupied}/{total} = {fraction:.6f}")


In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# === 假设你已有 mock_lens_data DataFrame ===
# 下面仅示例如何拼接所有样本点
kappa_all = np.concatenate([mock_lens_data['kappaA'], mock_lens_data['kappaB']])
gamma_all = np.concatenate([mock_lens_data['gammaA'], mock_lens_data['gammaB']])
s_all     = np.concatenate([mock_lens_data['sA'],     mock_lens_data['sB']])
samples = np.vstack([kappa_all, gamma_all, s_all]).T

# === Step 1: 定义最大边界立方体 ===
mins = samples.min(axis=0)
maxs = samples.max(axis=0)

# === Step 2: 分箱构建体素网格 ===
n_bins = 60
normed = (samples - mins) / (maxs - mins + 1e-10)
indices = np.floor(normed * n_bins).astype(int)
indices = np.clip(indices, 0, n_bins - 1)

# === Step 3: 统计体素占用 ===
voxels = np.zeros((n_bins, n_bins, n_bins), dtype=bool)
for i in indices:
    voxels[i[0], i[1], i[2]] = True

occupied = np.sum(voxels)
total = n_bins ** 3
fraction = occupied / total
print(f"[RESULT] 体素占比: {occupied}/{total} = {fraction:.6%}")

# === Step 4: 可视化体素分布 ===
x, y, z = np.where(voxels)
x_vals = mins[0] + (x + 0.5) * (maxs[0] - mins[0]) / n_bins
y_vals = mins[1] + (y + 0.5) * (maxs[1] - mins[1]) / n_bins
z_vals = mins[2] + (z + 0.5) * (maxs[2] - mins[2]) / n_bins

fig = go.Figure(data=[go.Scatter3d(
    x=x_vals, y=y_vals, z=z_vals,
    mode='markers',
    marker=dict(size=2, color=z_vals, colorscale='Viridis', opacity=0.7),
)])
fig.update_layout(
    scene=dict(
        xaxis_title='κ',
        yaxis_title='γ',
        zaxis_title='s',
    ),
    title=f"Occupied Voxels in κ–γ–s Space ({occupied}/{total} = {fraction:.4%})",
    margin=dict(l=0, r=0, b=0, t=40),
)
fig.show()


In [ ]:
import numpy as np
from scipy.spatial import ConvexHull
import plotly.graph_objects as go

# === Mock data: replace with actual mock_lens_data values ===
np.random.seed(42)
# mock_kappaA = np.random.uniform(0.2, 1.3, 500)
# mock_kappaB = np.random.uniform(0.3, 1.2, 500)
# mock_gammaA = np.random.uniform(0.2, 1.4, 500)
# mock_gammaB = np.random.uniform(0.3, 1.3, 500)
# mock_sA = np.random.uniform(0.01, 0.6, 500)
# mock_sB = np.random.uniform(0.02, 0.5, 500)
mock_kappaA = mock_lens_data['kappaA'].values
mock_kappaB = mock_lens_data['kappaB'].values
mock_gammaA = mock_lens_data['gammaA'].values
mock_gammaB = mock_lens_data['gammaB'].values
mock_sA     = mock_lens_data['sA'].values
mock_sB     = mock_lens_data['sB'].values

# === 拼接样本 ===
kappa_all = np.concatenate([mock_kappaA, mock_kappaB])
gamma_all = np.concatenate([mock_gammaA, mock_gammaB])
s_all     = np.concatenate([mock_sA,     mock_sB])
samples = np.vstack([kappa_all, gamma_all, s_all]).T

# === Convex hull ===
hull = ConvexHull(samples)

# === 体素网格 ===
n_bins = 60
mins = samples.min(axis=0)
maxs = samples.max(axis=0)
normed = (samples - mins) / (maxs - mins + 1e-10)
indices = np.floor(normed * n_bins).astype(int)
indices = np.clip(indices, 0, n_bins - 1)

voxels = np.zeros((n_bins, n_bins, n_bins), dtype=bool)
for i in indices:
    voxels[i[0], i[1], i[2]] = True

x, y, z = np.where(voxels)
x_vals = mins[0] + (x + 0.5) * (maxs[0] - mins[0]) / n_bins
y_vals = mins[1] + (y + 0.5) * (maxs[1] - mins[1]) / n_bins
z_vals = mins[2] + (z + 0.5) * (maxs[2] - mins[2]) / n_bins

# === 绘制图像 ===
fig = go.Figure()

# 体素点
fig.add_trace(go.Scatter3d(
    x=x_vals, y=y_vals, z=z_vals,
    mode='markers',
    marker=dict(size=2, color='blue', opacity=0.3),
    name='Occupied Voxels'
))

# 凸包边界
# === 凸包边界（单次绘制所有面）===
fig.add_trace(go.Mesh3d(
    x=samples[:, 0],
    y=samples[:, 1],
    z=samples[:, 2],
    i=hull.simplices[:, 0],
    j=hull.simplices[:, 1],
    k=hull.simplices[:, 2],
    color='red',
    opacity=0.2,
    name='Convex Hull',
    showscale=False
))


# # 最大立方体框线（bounding box）
# x0, y0, z0 = mins
# x1, y1, z1 = maxs
# cube_edges = [
#     [(x0,y0,z0), (x1,y0,z0)], [(x0,y0,z0), (x0,y1,z0)], [(x0,y0,z0), (x0,y0,z1)],
#     [(x1,y1,z1), (x0,y1,z1)], [(x1,y1,z1), (x1,y0,z1)], [(x1,y1,z1), (x1,y1,z0)],
#     [(x1,y0,z0), (x1,y1,z0)], [(x1,y0,z0), (x1,y0,z1)],
#     [(x0,y1,z0), (x1,y1,z0)], [(x0,y1,z0), (x0,y1,z1)],
#     [(x0,y0,z1), (x1,y0,z1)], [(x0,y0,z1), (x0,y1,z1)]
# ]
# for edge in cube_edges:
#     fig.add_trace(go.Scatter3d(
#         x=[edge[0][0], edge[1][0]],
#         y=[edge[0][1], edge[1][1]],
#         z=[edge[0][2], edge[1][2]],
#         mode='lines',
#         line=dict(color='black', width=2),
#         name='Bounding Box',
#         showlegend=False
#     ))

fig.update_layout(
    scene=dict(
        xaxis_title='κ',
        yaxis_title='γ',
        zaxis_title='s',
    ),
    title="κ–γ–s 空间中体素、凸包与最大边界盒",
    margin=dict(l=0, r=0, b=0, t=40),
    showlegend=True
)

fig.show()


In [ ]:
from scipy.spatial import Delaunay

# 1. 构造 Delaunay 网格（与 convex hull 同源）
hull = ConvexHull(samples)
delaunay = Delaunay(samples[hull.vertices])

# 2. 判断哪些点在 hull 内部
is_inside = delaunay.find_simplex(samples) >= 0

# 3. 输出统计
n_total = samples.shape[0]
n_inside = np.sum(is_inside)
print(f"样本点总数: {n_total}")
print(f"在凸包内部的点数: {n_inside}")
print(f"比例: {n_inside / n_total:.2%}")


In [10]:
from scipy.spatial import ConvexHull, Delaunay

# 原始样本
center = samples.mean(axis=0)  # 或用 (mins + maxs)/2 更对称
expand_factor = 1.4          # 扩大 5%

# 放大后的样本点
expanded_samples = center + (samples - center) * expand_factor

# 用扩大后的点构造新 hull
hull_expanded = ConvexHull(expanded_samples)
delaunay_expanded = Delaunay(expanded_samples[hull_expanded.vertices])

# 判断原始 samples 是否在扩大后的凸包中
is_inside_expanded = delaunay_expanded.find_simplex(samples) >= 0
print(f"被扩大凸包覆盖的比例: {np.mean(is_inside_expanded):.2%}")


被扩大凸包覆盖的比例: 100.00%


In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.spatial import ConvexHull

# === 构造样本点（替换成你自己的 samples）===
samples = np.vstack([
    kappa_all,
    gamma_all,
    s_all
]).T

# === 方法一：放大样本点后构造凸包 ===
center = samples.mean(axis=0)
scale = 1.8  # 放大5%
expanded_samples = center + (samples - center) * scale
hull_expanded = ConvexHull(expanded_samples)

# === 绘图: 原始点 + 扩大凸包 ===
fig = go.Figure()

# 原始样本点（蓝色）
fig.add_trace(go.Scatter3d(
    x=samples[:, 0], y=samples[:, 1], z=samples[:, 2],
    mode='markers',
    marker=dict(size=2, color='blue', opacity=0.3),
    name='Original Samples'
))

# 扩大的凸包边界（红色半透明）
for simplex in hull_expanded.simplices:
    fig.add_trace(go.Mesh3d(
        x=expanded_samples[simplex, 0],
        y=expanded_samples[simplex, 1],
        z=expanded_samples[simplex, 2],
        color='red',
        opacity=0.2,
        name='Expanded Convex Hull',
        showscale=False
    ))

# 轴与布局设置
fig.update_layout(
    scene=dict(
        xaxis_title='κ',
        yaxis_title='γ',
        zaxis_title='s'
    ),
    title="Expanded Convex Hull (Scaled Outward from Center)",
    margin=dict(l=0, r=0, b=0, t=40),
    showlegend=True
)

fig.show()


In [ ]:
import numpy as np
from scipy.spatial import ConvexHull
import plotly.graph_objects as go

# ==== 样本准备（替换为你的 mock_lens_data） ====
kappa_all = np.concatenate([mock_lens_data['kappaA'], mock_lens_data['kappaB']])
gamma_all = np.concatenate([mock_lens_data['gammaA'], mock_lens_data['gammaB']])
s_all     = np.concatenate([mock_lens_data['sA'],     mock_lens_data['sB']])
samples = np.vstack([kappa_all, gamma_all, s_all]).T

# ==== 原始凸包 ====
hull = ConvexHull(samples)

# ==== 外推每个三角面，按法向量扩展 ====
delta = 0.05  # 外推比例（5%）
expanded_points = []

for simplex in hull.simplices:
    pts = samples[simplex]
    # 计算法向量（两个边的叉积）
    v1 = pts[1] - pts[0]
    v2 = pts[2] - pts[0]
    normal = np.cross(v1, v2)
    norm_len = np.linalg.norm(normal)
    if norm_len == 0:
        continue
    normal /= norm_len

    # 平均点坐标，作为基点
    center = np.mean(pts, axis=0)
    outward_pts = pts + delta * normal  # 每个顶点沿法向量外推
    expanded_points.extend(outward_pts)

# ==== 合并新点 + 原始点再构造新凸包 ====
expanded_points = np.array(expanded_points)
all_points = np.vstack([samples, expanded_points])
expanded_hull = ConvexHull(all_points)

# ==== 可视化 ====
fig = go.Figure()

# 原始点
fig.add_trace(go.Scatter3d(
    x=samples[:, 0], y=samples[:, 1], z=samples[:, 2],
    mode='markers',
    marker=dict(size=2, color='purple', opacity=0.5),
    name='Samples'
))

# 原始凸包
for s in hull.simplices:
    fig.add_trace(go.Mesh3d(
        x=samples[s, 0],
        y=samples[s, 1],
        z=samples[s, 2],
        color='rgba(255,0,0,0.2)',
        opacity=0.2,
        showscale=False,
        name='Original Hull'
    ))

# 扩展后的凸包
for s in expanded_hull.simplices:
    fig.add_trace(go.Mesh3d(
        x=all_points[s, 0],
        y=all_points[s, 1],
        z=all_points[s, 2],
        color='rgba(0,100,255,0.15)',
        opacity=0.15,
        showscale=False,
        name='Expanded Hull'
    ))

fig.update_layout(
    scene=dict(
        xaxis_title='κ',
        yaxis_title='γ',
        zaxis_title='s',
    ),
    title=f"凸包法向量外推 {delta*100:.1f}%",
    margin=dict(l=0, r=0, b=0, t=50),
    showlegend=True
)

fig.show()



In [ ]:
import numpy as np
from scipy.spatial import cKDTree
import plotly.graph_objects as go

# === 准备数据 ===
kappa_all = np.concatenate([mock_lens_data['kappaA'], mock_lens_data['kappaB']])
gamma_all = np.concatenate([mock_lens_data['gammaA'], mock_lens_data['gammaB']])
s_all     = np.concatenate([mock_lens_data['sA'],     mock_lens_data['sB']])
samples = np.vstack([kappa_all, gamma_all, s_all]).T

# === 建立 KD 树加速最近距离计算 ===
tree = cKDTree(samples)

# === 定义一个包含所有点的网格 ===
k_min, g_min, s_min = samples.min(axis=0) - 0.2
k_max, g_max, s_max = samples.max(axis=0) + 0.2
grid_size = 50  # 分辨率

k_vals = np.linspace(k_min, k_max, grid_size)
g_vals = np.linspace(g_min, g_max, grid_size)
s_vals = np.linspace(s_min, s_max, grid_size)

kk, gg, ss = np.meshgrid(k_vals, g_vals, s_vals, indexing='ij')
grid_points = np.vstack([kk.ravel(), gg.ravel(), ss.ravel()]).T

# === 计算每个点到样本点集的最小距离 ===
dists, _ = tree.query(grid_points, k=1)
mask = (dists >= 0.1) & (dists <= 0.2)

surface_points = grid_points[mask]

# === 可视化 ===
fig = go.Figure()

# 数据点（可选）
fig.add_trace(go.Scatter3d(
    x=samples[:, 0], y=samples[:, 1], z=samples[:, 2],
    mode='markers', marker=dict(size=2, color='purple', opacity=0.5),
    name='Samples'
))

# 面上点
fig.add_trace(go.Scatter3d(
    x=surface_points[:, 0], y=surface_points[:, 1], z=surface_points[:, 2],
    mode='markers', marker=dict(size=1.5, color='orange', opacity=0.3),
    name='Shell: 0.1 ≤ d(x) ≤ 0.2'
))

fig.update_layout(
    scene=dict(
        xaxis_title='κ',
        yaxis_title='γ',
        zaxis_title='s',
    ),
    title="包裹在样本外的距离壳层",
    margin=dict(l=0, r=0, b=0, t=40),
    showlegend=True
)

fig.show()


In [ ]:
print(type(pt), pt.shape)


In [7]:
import numpy as np
from scipy.ndimage import convolve

# === Step 1: 生成网格 ===
voxel_size = 0.05
samples = np.vstack([
    np.concatenate([mock_lens_data['kappaA'], mock_lens_data['kappaB']]),
    np.concatenate([mock_lens_data['gammaA'], mock_lens_data['gammaB']]),
    np.concatenate([mock_lens_data['sA'],     mock_lens_data['sB']])
]).T

grid_min = samples.min(axis=0) - 0.1
grid_max = samples.max(axis=0) + 0.1
grid_shape = np.ceil((grid_max - grid_min) / voxel_size).astype(int)

# 位置 -> 索引
def coord_to_index(pt):
    return tuple(np.floor((pt - grid_min) / voxel_size).astype(int))

# 构建 occupancy grid（初始为 -99）
label_grid = np.full(grid_shape, fill_value=-99, dtype=int)

# Step 2: 设置样本点所在格点值为“样本数”（或设为0）
for pt in samples:
    idx = coord_to_index(pt)
    if all(0 <= i < s for i, s in zip(idx, grid_shape)):
        if label_grid[idx] < 0:
            label_grid[idx] = 1
        else:
            label_grid[idx] += 1  # 样本数叠加

# Step 3: 将正样本标记为 0，其余为 -99，准备扩散
label_grid[label_grid > 0] = 0

# Step 4: 构建 6邻域卷积核
kernel = np.zeros((3, 3, 3), dtype=int)
kernel[1, 1, 0] = kernel[1, 1, 2] = 1
kernel[1, 0, 1] = kernel[1, 2, 1] = 1
kernel[0, 1, 1] = kernel[2, 1, 1] = 1

# Step 5: 逐层扩展
max_layer = 100  # 最远标记距离层数
for layer in range(1, max_layer + 1):
    mask = (label_grid == -99)
    neighbors = convolve((label_grid == layer - 1).astype(int), kernel, mode='constant')
    new_layer = (neighbors > 0) & mask
    label_grid[new_layer] = layer

# label_grid 中：
# - 0 是原始样本点所在区域
# - 1 是第一圈包裹
# - 2 是第二圈...
# - -99 是未被包裹区域（即远处）

print("[INFO] 标记完成。各层格点数量：")
for i in range(max_layer + 1):
    print(f"  layer {i}: {(label_grid == i).sum()} voxels")


[INFO] 标记完成。各层格点数量：
  layer 0: 6381 voxels
  layer 1: 5891 voxels
  layer 2: 5947 voxels
  layer 3: 6014 voxels
  layer 4: 6054 voxels
  layer 5: 6237 voxels
  layer 6: 6489 voxels
  layer 7: 6609 voxels
  layer 8: 6654 voxels
  layer 9: 6688 voxels
  layer 10: 6678 voxels
  layer 11: 6645 voxels
  layer 12: 6666 voxels
  layer 13: 6582 voxels
  layer 14: 6589 voxels
  layer 15: 6608 voxels
  layer 16: 6624 voxels
  layer 17: 6596 voxels
  layer 18: 6608 voxels
  layer 19: 6628 voxels
  layer 20: 6636 voxels
  layer 21: 6648 voxels
  layer 22: 6661 voxels
  layer 23: 6682 voxels
  layer 24: 6705 voxels
  layer 25: 6547 voxels
  layer 26: 6470 voxels
  layer 27: 6378 voxels
  layer 28: 6283 voxels
  layer 29: 6181 voxels
  layer 30: 6074 voxels
  layer 31: 6005 voxels
  layer 32: 5933 voxels
  layer 33: 5855 voxels
  layer 34: 5765 voxels
  layer 35: 5671 voxels
  layer 36: 5576 voxels
  layer 37: 5474 voxels
  layer 38: 5368 voxels
  layer 39: 5266 voxels
  layer 40: 5205 voxels
  laye

In [9]:
from itertools import product

# ==== 参数 ====
layers_to_sample = 1
n_per_dim = 4  # 每个 voxel 维度上的采样点数（即每个 voxel 有 n^3 个点）
half_size = voxel_size / 2

# ==== 1. 找出 0~layers_to_sample 层的 voxel 索引 ====
voxel_indices = np.argwhere((label_grid >= 0) & (label_grid <= layers_to_sample))

print(f"[INFO] 待采样 voxel 总数: {len(voxel_indices)}")

# ==== 2. 定义每个 voxel 内的网格采样 ====
offsets = np.linspace(-half_size, half_size, n_per_dim)
offset_grid = np.array(list(product(offsets, offsets, offsets)))  # shape=(n^3, 3)

# ==== 3. 转换为真实空间坐标 ====
all_sample_points = []

for idx in voxel_indices:
    center = grid_min + (idx + 0.5) * voxel_size
    points = center + offset_grid  # 每个 voxel 中的 n^3 个点
    all_sample_points.append(points)

# 合并所有点
all_sample_points = np.vstack(all_sample_points)  # shape=(N_voxels * n^3, 3)

print(f"[INFO] 总采样点数: {len(all_sample_points)}")


[INFO] 待采样 voxel 总数: 12272
[INFO] 总采样点数: 785408


In [13]:
# 所有体素总数
total_voxels = label_grid.size

# 第0层（含样本点的体素）数量
layer0_voxels = np.sum(label_grid == 0)

# 占比
fraction = layer0_voxels / total_voxels
print(f"[RESULT] 样本体素占总网格比例: {layer0_voxels}/{total_voxels} = {fraction:.6%}")


[RESULT] 样本体素占总网格比例: 6381/501009 = 1.273630%


In [14]:
# === Step 1: 定义限定范围 ===
kappa_range = (0.0, 1.5)
gamma_range = (0.0, 1.5)
s_range     = (0.0, 1.0)

# 将坐标范围转换为 index 范围
def coord_to_grid_index_range(coord_min, coord_max, grid_min, voxel_size):
    i_min = int(np.floor((coord_min - grid_min) / voxel_size))
    i_max = int(np.ceil((coord_max - grid_min) / voxel_size))
    return max(i_min, 0), min(i_max, label_grid.shape[0])  # 防止越界

ix_min, ix_max = coord_to_grid_index_range(*kappa_range, grid_min[0], voxel_size)
iy_min, iy_max = coord_to_grid_index_range(*gamma_range, grid_min[1], voxel_size)
iz_min, iz_max = coord_to_grid_index_range(*s_range,     grid_min[2], voxel_size)

# === Step 2: 截取子网格 ===
subgrid = label_grid[ix_min:ix_max, iy_min:iy_max, iz_min:iz_max]

# === Step 3: 统计第0层在该范围内的占比 ===
total_voxels = subgrid.size
layer0_voxels = np.sum(subgrid == 0)
fraction = layer0_voxels / total_voxels

print(f"[限定范围] κ ∈ [0,1.5], γ ∈ [0,1.5], s ∈ [0,1]")
print(f"[RESULT] 第0层体素占比: {layer0_voxels}/{total_voxels} = {fraction:.6%}")


[限定范围] κ ∈ [0,1.5], γ ∈ [0,1.5], s ∈ [0,1]
[RESULT] 第0层体素占比: 3782/17577 = 21.516755%


In [ ]:
import plotly.graph_objects as go

def plot_layer(layer_id):
    coords = np.argwhere(label_grid == layer_id)
    real_coords = grid_min + (coords + 0.5) * voxel_size
    fig = go.Figure(data=[go.Scatter3d(
        x=real_coords[:,0], y=real_coords[:,1], z=real_coords[:,2],
        mode='markers', marker=dict(size=2, color='blue', opacity=0.5),
        name=f'Layer {layer_id}'
    )])
    fig.update_layout(scene=dict(xaxis_title='κ', yaxis_title='γ', zaxis_title='s'),
                      title=f"Layer {layer_id} Structure",
                      margin=dict(l=0, r=0, b=0, t=40))
    fig.show()


In [ ]:
plot_layer(50)  # 绘制第0层（样本点所在层）

In [ ]:
import plotly.graph_objects as go

def get_voxel_edges(center, size):
    """
    给定立方体中心坐标和边长，返回 12 条边的两个端点（24个点，12条边）
    """
    cx, cy, cz = center
    d = size / 2
    # 8个顶点
    pts = np.array([
        [cx-d, cy-d, cz-d],
        [cx-d, cy-d, cz+d],
        [cx-d, cy+d, cz-d],
        [cx-d, cy+d, cz+d],
        [cx+d, cy-d, cz-d],
        [cx+d, cy-d, cz+d],
        [cx+d, cy+d, cz-d],
        [cx+d, cy+d, cz+d],
    ])
    # 12条边的端点索引
    edges = [
        (0,1), (0,2), (0,4),
        (1,3), (1,5),
        (2,3), (2,6),
        (3,7),
        (4,5), (4,6),
        (5,7),
        (6,7)
    ]
    lines = []
    for i,j in edges:
        lines.append((pts[i], pts[j]))
    return lines

def plot_voxel_wireframes(label_grid, target_layer, grid_min, voxel_size, max_voxels=6000):
    coords = np.argwhere(label_grid == target_layer)
    if len(coords) > max_voxels:
        print(f"[INFO] 太多立方体 ({len(coords)}), 仅绘制前 {max_voxels} 个")
        coords = coords[:max_voxels]

    fig = go.Figure()
    for c in coords:
        center = grid_min + (c + 0.5) * voxel_size
        for pt1, pt2 in get_voxel_edges(center, voxel_size):
            fig.add_trace(go.Scatter3d(
                x=[pt1[0], pt2[0]],
                y=[pt1[1], pt2[1]],
                z=[pt1[2], pt2[2]],
                mode='lines',
                line=dict(color='blue', width=1),
                showlegend=False
            ))

    fig.update_layout(
        scene=dict(
            xaxis_title='κ',
            yaxis_title='γ',
            zaxis_title='s',
        ),
        title=f"Layer {target_layer} - Wireframe Cubes",
        margin=dict(l=0, r=0, b=0, t=40),
        showlegend=False
    )
    fig.show()
